Given the names and grades for each student in a class of  students, store them in a nested list and print the name(s) of any student(s) having the second lowest grade.

In [1]:
from bs4 import BeautifulSoup, element
import pandas as pd
import numpy as np
import requests
import time
import unidecode
from user_agent import generate_user_agent
from proxies_gen import get_proxies, test_proxies
from itertools import cycle
from lxml.html import fromstring
from multiprocessing import Pool, cpu_count  # This is a thread-based Pool
from requests.exceptions import ConnectionError, Timeout, ProxyError, RequestException
from urllib3.exceptions import ProtocolError
import sys
import os
sys.setrecursionlimit(10000)  # need to optimize code.
proxy_enabled = True


def parse_games(game_tags):
    """
    parse the games table on current page
    parameters:
    game_tags: games tags after reading the html page
    df: the dataframe where we will store the games
    """
    global rec_count
    global df
    for tag in game_tags:
        game = {}
        game["Name"] = " ".join(tag.string.split())
        #print(rec_count+1, 'Fetch Data for game', unidecode.unidecode(game['Name']))

        data = tag.parent.parent.find_all("td")
        if data:
            game["Rank"] = np.int32(data[0].string)
            game["img_url"] = data[1].a.img.get('src')
            game["url"] = data[2].a.get('href')
            if len(game["Name"].split("/")) > 1:
                # replace accented chars with ascii
                game["basename"] = unidecode.unidecode(
                    game['Name'].strip().split('/')[0].strip().replace(' ', '-'))
            else:
                game["basename"] = game["url"].rsplit('/', 2)[1]
            game["Platform"] = data[3].img.get('alt')
            game["Publisher"] = data[4].get_text().strip()
            game["Developer"] = data[5].get_text().strip()
            game["Vgchartzscore"] = data[6].get_text().strip()
            game["Critic_Score"] = float(
                data[7].string) if not data[7].string.startswith("N/A") else np.nan
            game["User_Score"] = float(
                data[8].string) if not data[8].string.startswith("N/A") else np.nan
            game["Total_Shipped"] = float(
                data[9].string[:-1]) if not data[9].string.startswith("N/A") else np.nan
            game["Global_Sales"] = float(
                data[10].string[:-1]) if not data[10].string.startswith("N/A") else np.nan
            game["NA_Sales"] = float(
                data[11].string[:-1]) if not data[11].string.startswith("N/A") else np.nan
            game["EU_Sales"] = float(
                data[12].string[:-1]) if not data[12].string.startswith("N/A") else np.nan
            game["JP_Sales"] = float(
                data[13].string[:-1]) if not data[13].string.startswith("N/A") else np.nan
            game["Other_Sales"] = float(
                data[14].string[:-1]) if not data[14].string.startswith("N/A") else np.nan
            year = data[15].string.split()[-1]
            if year.startswith('N/A'):
                game["Year_of_Release"] = 'N/A'
            else:
                if int(year) >= 70:
                    year_to_add = np.int32("19" + year)
                else:
                    year_to_add = np.int32("20" + year)
                game["Year_of_Release"] = year_to_add
            game["Last_Update"] = data[16].get_text().strip()
            game['Genre'] = 'N/A'
            game['Rating'] = 'N/A'
            game['status'] = 0
            df = df.append(game, ignore_index=True)
        rec_count += 1


def parse_genre_esrb(df):
    """loads every game's url to get genre and esrb rating"""
    headers = {'User-Agent': generate_user_agent(
        device_type='desktop', os=('mac', 'windows'))}
    proxy = {}
    if proxy_enabled:
        #print("\n******getting list of proxies and testing them******'\n")
        # this an api call which returns a list of working proxies that get checked evrey 15 minutes
        proxies = cycle(get_proxies(5))
        proxy = next(proxies)

    for index, row in df.iterrows():
        try:
            game_page = requests.get(df.at[index, 'url'], headers=headers, proxies={"http": proxy, "https": proxy}, timeout=5)
            if game_page.status_code == 200:
                sub_soup = BeautifulSoup(game_page.text, "lxml")
                # again, the info box is inconsistent among games so we
                # have to find all the h2 and traverse from that to the genre
                gamebox = sub_soup.find("div", {"id": "gameGenInfoBox"})
                h2s = gamebox.find_all('h2')
                # make a temporary tag here to search for the one that contains
                # the word "Genre"
                temp_tag = element.Tag
                for h2 in h2s:
                    if h2.string == 'Genre':
                        temp_tag = h2
                df.loc[index, 'Genre'] = temp_tag.next_sibling.string

                # find the ESRB rating
                game_rating = gamebox.find('img').get('src')
                if 'esrb' in game_rating:
                    df.loc[index, 'Rating'] = game_rating.split(
                        '_')[1].split('.')[0].upper()
                # we successfuly got the genre and rating
                df.loc[index, 'status'] = 1
                #print('Successfully scraped genre and rating for :', df.at[index, 'Name'])

        except(ProxyError):
            proxy = next(proxies)

        except (ConnectionError, Timeout, ProtocolError, TimeoutError):
            #print('Something went wrong while connecting to', df.at[index, 'Name'], 'url, will try again later')
            continue

        except Exception as e:
            #print('different error occurred while connecting, will pass')
            continue
        # wait for 1 seconds between every call,
        # we do not want to get blocked or abuse the server
        time.sleep(1)
    return df


def retry_game(df):
    """try to scrape the missing data again"""
    return parse_genre_esrb(df)


if __name__ == "__main__":
    def process_games(df):
        failed_games = len(df[df['status'] == 0])
        NUM_WORKERS = cpu_count() * 2
        df_subsets = np.array_split(df[df['status'] == 0], NUM_WORKERS)
        #update num_workers
        df_subsets = [i for i in df_subsets if len(i) != 0]
        if len(df_subsets) != 0:
            NUM_WORKERS = len(df_subsets)# we don't want to have a worker for empty subsets
            pool = Pool(processes=NUM_WORKERS)
            results = pool.map(retry_game, df_subsets)
            try:
                df_updated = pd.concat(results)
                df = pd.concat([df[df['status'] == 1], df_updated])
            except: 
                print('error occurred while joining dataframe')
            pool.close()
            pool.join()
        return df

    rec_count = 0
    start_time = time.time()
    current_time = time.time()
    crashed_tag = 'before_crashing_'
    exists = [s for s in os.listdir() if crashed_tag in s]
    if exists:
        print("found a data saved from a crash, will continue on it")
        csvfilename = exists[0].replace(crashed_tag, '')
        df = pd.read_csv(exists[0])
        rec_count = df['Rank'].max()
        page = int(rec_count/1000) + 1 # because we already scraped current 
        df = process_games(df)
    else:
        csvfilename = "vgsales-" + time.strftime("%Y-%m-%d_%H_%M_%S") + ".csv"

    # initialize a panda dataframe to store all games with the following columns:
    # rank, name, img-url, vgchartz score, genre, ESRB rating, platform, developer,
    # publisher, release year, critic score, user score, na sales, pal sales,
    # jp sales, other sales, total sales, total shipped, last update, url, status
    # last two columns for debugging
    if not exists:
        df = pd.DataFrame(columns=[
            'Rank', 'Name', 'basename', 'Genre', 'Rating', 'Platform', 'Publisher',
            'Developer', 'VGChartz_Score', 'Critic_Score', 'User_Score',
            'Total_Shipped', 'Global_Sales', 'NA_Sales', 'EU_Sales', 'JP_Sales',
            'Other_Sales', 'Year_of_Release', 'Last_Update', 'url', 'status'])

    urlhead = 'http://www.vgchartz.com/games/games.php?page='
    urltail = '&results=1000&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership=Both'
    urltail += '&banner=Both&showdeleted=&region=All&goty_year=&developer='
    urltail += '&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1'
    urltail += '&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1'
    urltail += '&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=Yes&showgenre=1'

    # get the number of pages
    vglink = requests.get('http://www.vgchartz.com/gamedb/').text
    x = fromstring(vglink).xpath(
        "//th[@colspan='3']/text()")[0].split('(', 1)[1].split(')')[0]
    pages = int(x.split(',')[0])

    if not exists: page = 1
    while True:
        if page > pages:
            break
        try:
            proxy = get_proxies(1)[0]
            headers = {'User-Agent': generate_user_agent(
                device_type='desktop', os=('mac', 'windows'))}
            surl = urlhead + str(page) + urltail
            r = requests.get(surl, headers=headers, proxies={
                            'http': proxy, 'https': proxy}, timeout=10)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'lxml')
                print("******Scraping page " + str(page) + "******'\n")

                # vgchartz website is really weird so we have to search for
                # <a> tags with game urls
                game_tags = list(filter(
                    lambda x: x.attrs['href'].startswith('http://www.vgchartz.com/game/'), soup.find_all("a")))[10:]
                # discard the first 10 elements because those
                # links are in the navigation bar

                parse_games(game_tags)
                page += 1
                print('\n******begin scraping for Genre and Rating******\n')
                df = process_games(df)

        except (ConnectionError, Timeout, ProxyError, RequestException, ProtocolError, TimeoutError):
            print('Something went wrong while connecting to page: ',
                page, ', will try again later')
            #proxy = get_proxies(1)
            time.sleep(10)

        except Exception as e:
            print("something went wrong! We're on page: " +
                str(page) + '\nSaving successfully crawled data')
            print("Exception: ", e)
            df.to_csv(crashed_tag + csvfilename, sep=",",
                    encoding='utf-8', index=False)
            raise e


    failed_games = len(df[df['status'] == 0])
    print("******Finished scraping games, will try to scrape missing data******")
    # 36 hours max, should be enough to scrape everything
    t_end = start_time + 60 * 60 * 36
    while True:
        try:
            df = process_games(df)
            failed_games = len(df[df['status'] == 0])
            if failed_games == 0 or time.time() > t_end:
                break
            #print('Number of not scraped yet:', failed_games, '\n')
            time.sleep(10)  # wait for 10 seconds for the server to recover?
        except Exception as e:
            print("something went wrong! We're on page: " + str(page) + '\nSaving successfully crawled data')
            print("Exception: ", e)
            df.to_csv(crashed_tag + csvfilename, sep=",",
                    encoding='utf-8', index=False)
            raise e

    elapsed_time = time.time() - start_time
    print("Scraped", rec_count, "games in", round(elapsed_time/60, 2), "minutes.")

    # select only these columns in the final dataset
    df = df.sort_values(by=['Rank'])
    df.to_csv('complete-vgchartz.csv', sep=",", encoding='utf-8', index=False)
    df_final = df[[
        'Rank', 'Name', 'Platform', 'Year_of_Release', 'Genre', 'Rating',
        'Publisher', 'Developer', 'Critic_Score', 'User_Score',
        'Global_Sales', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]

    df_final.to_csv(csvfilename, sep=",", encoding='utf-8', index=False)
    print("Wrote scraper data to", csvfilename)

ModuleNotFoundError: No module named 'proxies_gen'

In [2]:
from lxml.html import fromstring
import requests
import numpy as np
from itertools import cycle


def get_proxies(num=None):
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = list(requests.get('https://proxy.rudnkh.me/txt').text.split())
    for i in parser.xpath('//tbody/tr'):
         if i.xpath('.//td[7][contains(text(),"yes")]'):
                proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
                proxies.append(proxy)

    link = "https://api.proxyscrape.com/?request=getproxies&proxytype=http&timeout=1000&country=all&ssl=all&anonymity=all&uptime=100"
    proxies = list(requests.get(link).text.split())
    np.random.shuffle(proxies)
    proxies = []
    if len(proxies) == 0:
        proxies = list(requests.get(
            link[:-3]+'99').text.split())  # change uptime to 99
        np.random.shuffle(proxies)
    # print('Found', len(proxies), 'proxies, testing them now')

    if num is None:
        num = len(proxies)
    tested = test_proxies(proxies, num)
    return tested


def test_proxies(proxies, num):
    url = 'https://httpbin.org/ip'
    proxy_pool = cycle(proxies)
    working_proxies = []
    for i in range(1, len(proxies)):
        if num == 0:
            break
        # Get a proxy from the pool
        proxy = next(proxy_pool)
        # print("Request #%d" % i)
        try:
            response = requests.get(
                url, proxies={"http": proxy, "https": proxy}, timeout=1)
            # print(response.json())
            working_proxies.append(proxy)
            num -= 1
        except:
            # Most free proxies will often get connection errors. You will have retry the entire request using another proxy to work.
            # print("Skipping. Connnection error")
            pass
    return working_proxies


proxies = get_proxies(5)
with open('proxies.txt') as f:
        proxies = f.read().splitlines()
test_proxies(proxies, 10)
print(proxies)

ConnectionError: HTTPSConnectionPool(host='proxy.rudnkh.me', port=443): Max retries exceeded with url: /txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000218C7C94550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [3]:
def find_console_tags(soup):
    # Console tags are stored as images, so we find the image tag and record its 'alt' value as text
    consoles = list()
    for img in soup.find_all('img'):
        if 'images/consoles'in img['src']:
            # Cut file path elements from string
            console_tag = (img['src'][17:-6])
            consoles.append(img['alt'])
    return consoles


# Find the names of games from the links
def find_names_column(table_path):
    names_list = list()
    for row in table_path.xpath('.//tr'):
        for td in row.xpath('.//td'):
            if not td.find('a') is None:
                names_list.append(td.find('a').text.strip()) 
    return names_list

# Write a function that takes in a VGChartz URL and gives us all the data in their video game database
def scrape_vgchartz_videogame_db_page(url):
    
    response = requests.get(url)

    ### Check the Status
    assert(response.status_code == 200)," Website not OK " # status code = 200 => OK
    
    #Store the contents of the website under doc
    page=response.text
    soup = BeautifulSoup(page, "lxml")
    doc = lh.fromstring(response.content)
    
    # Selects the table with all the data in it on HTML using xpath
    target_table_path = doc.xpath('//*[@id="generalBody"]/table')[0]

    # Find column values that won't be scraped correctly with .text option
    names_list = find_names_column(target_table_path)
    consoles = find_console_tags(soup)
    
    # Parse non-image and non-URL info from the data table to a pandas DataFrame
    row_dict={}
    df=pd.DataFrame()
    row_list= list()
    for counter,row in enumerate(target_table_path.xpath(".//tr")):
        if counter > 2: # To skip header rows
            row_list=[td.text for td in row.xpath(".//td")]
            row_dict[counter] = row_list

    df=pd.DataFrame.from_dict(row_dict).transpose()
    df.columns = ['position','game','blank','console','publisher','developer','vgchart_score',\
                 'critic_score','user_score','total_shipped','total_sales',\
                  'na_sales','pal_sales','japan_sales','other_sales',\
                  'release_date','last_update']
    
    # Correct the console and game columns using scraped values
    
    df=df.reset_index().drop(columns = ['index','blank'])
    df['console'] = consoles
    df['game'] = names_list
    return df

    # We can 'hack' the URL to display any number of results per page. I'll leave it as an argument.
def scrape_all_vg_chartz_videogame_db(results_per_page):
    df = pd.DataFrame()
    current_page = 1
    games_left = True
    while games_left:
        url = 'http://www.vgchartz.com/games/games.php?page=' + str(current_page) +\
        '&results=' + str(results_per_page) + '&name=&console=&keyword=&publisher=&genre=&order=Sales&ownership\
        =Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction\
        =DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&\
        showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&\
        showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'
        new_df = scrape_vgchartz_videogame_db_page(url)
        df = df.append(new_df)

# Run the code to scrape! I did 10,000 rows per page to speed things up.
df=scrape_all_vg_chartz_videogame_db(10000)

# Compress and store for later!
df.to_csv(vgsales.csv, sep=",", encoding='utf-8', index=False)
print("Wrote scraper data to", csvfilename)

NameError: name 'lh' is not defined

In [4]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import re
import math
import scrapy
import json
import logging
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
from timeit import default_timer as timer

# Here is our JSON writer pipeline

class JsonWriterPipeline(object):

    # When the spider is open, it writes itself to the gamesresult to a julia file
    def open_spider(self, spider):
        self.file = open('gamesresult.jl', 'w')

    # When the spider closes, it closes that file as it is done writing to it
    def close_spider(self, spider):
        self.file.close()

    # This function dictates how the spider writes to the .jl file
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

# I'll just create a string for today's date to append to the end of the names of the files we create
today_date_string = str(date.today().month) + "_" + str(date.today().day) + "_" + str(date.today().year)

page = 2 # The first page we will need to jump to is page number 2, so this is that variable
genre = 0 # We are starting at the first genre in the list

# The quick and admittedly dirty way to do this is if we want to know the genre of each game is to cycle through the urls with
# each of the following genres as a way of changing the genre parameter of the web address

genre_list = ["Action",
             "Adventure",
             "Action-Adventure",
             "Board+Game",
             "Education",
             "Fighting",
             "Misc",
             "MMO",
             "Music",
             "Party",
             "Platform",
             "Puzzle",
             "Racing",
             "Role-Playing",
             "Sandbox",
             "Shooter",
             "Simulation",
             "Sports",
             "Strategy",
             "Visual+Novel"]

class VGSpider(scrapy.Spider):
    global genre

    name = "game_spider"
    start_urls = ['https://www.vgchartz.com/games/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=' + genre_list[0] + '&boxart=Both&banner=Both&ownership=Both&showmultiplat=No&results=200&order=Sales&showtotalsales=0&showtotalsales=1&showpublisher=0&showpublisher=1&showvgchartzscore=0&showvgchartzscore=1&shownasales=0&shownasales=1&showdeveloper=0&showdeveloper=1&showcriticscore=0&showcriticscore=1&showpalsales=0&showpalsales=1&showreleasedate=0&showreleasedate=1&showuserscore=0&showuserscore=1&showjapansales=0&showjapansales=1&showlastupdate=0&showlastupdate=1&showothersales=0&showothersales=1&showshipped=0&showshipped=1']

    # Here's where we set the logging settings.
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # This uses the functions in the JsonWriterPipeline class to write the
                                                              # Julia file
        'FEED_FORMAT':'json',                                 # This sets the feed exporter to export as a JSON file
        'FEED_URI': "gamesresult-" + today_date_string + ".json" # This simply sets the title for said JSON file
    }

    def parse(self, response):

        global genre
        global page

        # Here we declare the selector for each piece of data--this tells scrapy where to check for each item

        IMAGE_SELECTOR = './/td[2]/div/a/div/img/@src'
        TITLE_SELECTOR = './/td[3]/a/text()'
        CONSOLE_SELECTOR = './/td[4]/img/@alt'
        PUBLISHER_SELECTOR = './/td[5]/text()'
        DEVELOPER_SELECTOR = './/td[6]/text()'
        VGSCORE_SELECTOR = './/td[7]/text()'
        CRITIC_SELECTOR = './/td[8]/text()'
        USER_SELECTOR = './/td[9]/text()'
        TOTALSHIPPED_SELECTOR = './/td[10]/text()'
        TOTALSALES_SELECTOR = './/td[11]/text()'
        NASALES_SELECTOR = './/td[12]/text()'
        PALSALES_SELECTOR = './/td[13]/text()'
        JPSALES_SELECTOR = './/td[14]/text()'
        OTHER_SELECTOR = './/td[15]/text()'
        RELEASEDATE_SELECTOR = './/td[16]/text()'
        UPDATE_SELECTOR = './/td[17]/text()'

        # We loop through each row (so each game) in the table, and snag the data we want, giving each one a name that make sense

        for row in response.xpath('//*[@id="generalBody"]/table[1]/tr'):
            yield {

                'img' : row.xpath(IMAGE_SELECTOR).extract(),
                'title' : row.xpath(TITLE_SELECTOR).extract(),
                'console' : row.xpath(CONSOLE_SELECTOR).extract(),
                'publisher' : row.xpath(PUBLISHER_SELECTOR).extract(),
                'developer' : row.xpath(DEVELOPER_SELECTOR).extract(),
                'vg_score' : row.xpath(VGSCORE_SELECTOR).extract(),
                'critic_score' : row.xpath(CRITIC_SELECTOR).extract(),
                'user_score' : row.xpath(USER_SELECTOR).extract(),
                'total_shipped' : row.xpath(TOTALSHIPPED_SELECTOR).extract(),
                'total_sales' : row.xpath(TOTALSALES_SELECTOR).extract(),
                'na_sales' : row.xpath(NASALES_SELECTOR).extract(),
                'pal_sales' : row.xpath(PALSALES_SELECTOR).extract(),
                'jp_sales' : row.xpath(JPSALES_SELECTOR).extract(),
                'other_sales' : row.xpath(OTHER_SELECTOR).extract(),
                'release_date' : row.xpath(RELEASEDATE_SELECTOR).extract(),
                'last_update' : row.xpath(UPDATE_SELECTOR).extract(),
                'genre' : genre_list[genre]
            }

        # next_page puts together--you guessed it--the next page. It uses the page number we established at the beginning
        # and the genre parameter we're on currently.

        next_page = "https://www.vgchartz.com/games/games.php?page=" + str(page) + "&results=200&name=&console=&keyword=&publisher=&genre="+ genre_list[genre] + "&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No"

        # This selector will help us know just how far to go by grabbing the number of results for the given query total, not
        # just on the page

        RESULTS_SELECTOR = '//*[@id="generalBody"]/table[1]/tr[1]/th[1]/text()'

        # Below we grab that figure which is a string, and extract the numbers from it using a regular expression

        results = response.xpath(RESULTS_SELECTOR).extract_first()
        results_pat = r'([0-9]{1,9})'
        results = results.replace(",", "")

        # We then can divide that number by 200 (the number of results per page), and thusly figure out just how many pages we
        # should cycle through before we've reached the end and should either switch genres or finish up.

        last_page = math.ceil(int(re.search(results_pat, results).group(1)) / 200)

        # Below we test whether we are at the last page and whether we've reached the last page of the last genre
        # If we've reached the last page and we've reached the last genre (which according to our list is Visual Novel)
        # we can end out spider and close up shop.

        if (page > last_page) & (genre_list[genre] == "Visual+Novel"):
            print(genre_list[genre])
            return "All done!"

        # If we've reached the last page, but not the last genre (anything BUT Visual Novel), we'll reset our page, move onto the
        # next genre and keep scraping.
        elif (page > last_page) & (genre_list[genre] != "Visual+Novel"):
            print(genre_list[genre])
            page = 1
            genre += 1
            next_page = "https://www.vgchartz.com/games/games.php?page=" + str(page) + "&results=200&name=&console=&keyword=&publisher=&genre="+ genre_list[genre] + "&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No"
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
                )
            page += 1

        # If we haven't reached the last page at all, we can just keep going without changing the genre parameter, we'll just have
        # to move onto the next page
        elif next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
                )
            time.sleep(3)
            page += 1

# Here we finally simply create the process and set the spider we've created to crawl away
if __name__ == "__main__":
    process = CrawlerProcess(get_project_settings())
    start = timer()
    process.crawl(VGSpider)
    process.start(stop_after_crawl=True) # Blocks here until the crawl is finished
    end = timer()
    print("It took " + str(end - start) + " seconds to retrieve this data.")

    # And then we can read in the JSON file we've created

    games = pd.read_json("gamesresult-" + today_date_string + ".json")

    # Now we'll clean it up!
    # The webcrawler pulled three blank rows for each page because of the structure of the table, so let's go ahead
    # and filter those out. The crawler also pulled everything as lists with a single element due to the JSON, so
    # we'll be checking the length of each element to see if there's anything inside, if not we'll toss it.

    games = games[~(games["title"].str.len() == 0)]

    # Next we'll need to convert our single-element lists to the elements themselves, so we'll select the first element
    # in each list to be the actual value we want in each cell.

    for column in ['console', 'critic_score', 'developer', 'img', 'jp_sales',
           'last_update', 'na_sales', 'other_sales', 'pal_sales', 'publisher',
           'release_date', 'title', 'total_sales', 'total_shipped', 'user_score',
           'vg_score']:
        games[column] = games[column].apply(lambda x : x[0])

    # There are also some trailing spaces on some of the columns, so let's go ahead and trim those off with str.strip()

    games = games.apply(lambda x : x.str.strip())

    # If we hadn't done the previous step, this wouldn't have worked, but now we can convert all "N/A" strings in the
    # dataset to numpy nan values.

    games = games.replace("N/A", np.nan)

    # Now we'll start cleaning the individual columns. I'll first write a function to clean each numerical column, since
    # they'll all follow the same general rules.

    def clean_nums(column, dataframe):
        dataframe[column] = dataframe[column].str.strip("m") # This will strip the "m" off the end of each string if it's there
        dataframe[column] = dataframe[column].apply(lambda x : float(x)) # This will turn all the values from strings to floats

    # Now we'll just apply it to all of the sales columns

    sales_columns = ["na_sales",
                     "jp_sales",
                     "pal_sales",
                     "other_sales",
                     "total_sales",
                     "total_shipped",
                     "vg_score",
                     "user_score",
                     "critic_score"]

    for column in sales_columns:
        clean_nums(column, games)

    # Next we'll need to clean up the dates which are in string format. We'll use some regex to extract the information
    # and then convert to datetime objects.

    day_pat = r"([0-9]{2})(?=[a-z]{2})" # A regex pattern to select only the numerical day of the month
    month_pat = r"([A-Z][a-z]{2})" # A regex pattern to select only the string abbreviated month
    year_pat = r"([0-9]{2}(?![a-z]{2}))" # A regex pattern to select only the numerical year

    # And here is a month string to month integer translation dictionary.

    month_to_num = {'Sep' : 9,
                    'Jul' : 7,
                    'Oct' : 10,
                    'Mar' : 3,
                    'Dec' : 12,
                    'Feb' : 2,
                    'Nov' : 11,
                    'Jun' : 6,
                    'Aug' : 8,
                    'May' : 5,
                    'Apr' : 4,
                    'Jan' : 1
                    }

    # Now we're actually ready to create the function to clean the dates.

    def clean_dates(text):
        global day_pat
        global month_pat
        global year_pat
        global month_to_num
        if text is np.nan:
            return text

        day = int((re.search(day_pat, text).group(1))) # This one is the easiest, we extract the numbers and convert to integer
        month = month_to_num[(re.search(month_pat, text).group(1))] # This one we use the month string to look up the integer in our dictionary
        year = (re.search(year_pat, text).group(1)) # Year will require a little more work as we need to fill in the first two digits

        # The best we can do here is see how high the first integer is, and if it's greater than seven, we can safely assume
        # (for now) that it was from the nineties. Otherwise, it's in the 2000s. That will be alright before 2070 in which case
        # we've got a classic y2k situation ready to ruin us.

        if int(year[0]) >= 7:
            year = int("19" + year)
        else:
            year = int("20" + year)

        return(datetime.datetime(year, month, day))

    # We will apply the date cleanup function across our two date columns.

    for column in ["last_update", "release_date"]:
        games[column] = games[column].apply(lambda x : clean_dates(x))

    # A quick replacement of the +'s used in the url for genre to make our genre column more readable

    games["genre"] = games["genre"].str.replace("+", " ")

    # In exploring the data, I noticed there were a few dates being used
    # as placeholders. For the longevity of this code, I will not exclude
    # 12/31/2020 and 12/31/2021, but any user using this code before these dates
    # should be careful to not rely on them, and explore to see if they are
    # still being used as placeholders. However, 1/1/1970 is used as a
    # placeholder, and is the only date to show up in 1970, thus I can replace
    # these dates with a nan value while keeping our mind at ease.

    games.loc[games["release_date"].dt.year == 1970, "release_date"] = np.nan 

    # Finally, let's reorder the columns in a way that makes the most sense.

    games = games[["img", "title", "console", "genre", "publisher", "developer", "vg_score", "critic_score", "user_score", "total_shipped", "total_sales", "na_sales", "jp_sales", "pal_sales", "other_sales", "release_date", "last_update"]]

    # And now we have a delightful little clean dataframe of videogames that's as current as possible!

    games.to_csv("vgchartz-" + today_date_string +".csv", index=False)

    print(str(games["title"].count()) + " game records retrieved." )
    print("File saved as vgchartz-" + today_date_string + ".csv")

2022-10-15 20:56:03 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-10-15 20:56:03 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.2.0, Python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 21.0.0 (OpenSSL 1.1.1q  5 Jul 2022), cryptography 3.4.8, Platform Windows-10-10.0.22000-SP0
2022-10-15 20:56:04 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}
2022-10-15 20:56:09 [py.warnings] WARNING: C:\Users\janat\anaconda\lib\site-packages\scrapy\extensions\feedexport.py:289: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2022-10-15 20:56:24 [py.warnings] WARNING: C:\Users\janat\anaconda\lib\site-packages\scrapy\core\scraper.py:157: UserWarning: The "VGSpider.parse" method is a generator and includes a "ret

Action


2022-10-15 21:00:03 [scrapy.core.downloader.handlers.http11] WARNING: Got data loss in https://www.vgchartz.com/games/games.php?page=4&results=200&name=&console=&keyword=&publisher=&genre=Adventure&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No. If you want to process broken responses set the setting DOWNLOAD_FAIL_ON_DATALOSS = False -- This message won't be shown in further requests


Adventure
Action-Adventure
Board+Game
Education
Fighting
Misc
MMO
Music
Party
Platform
Puzzle
Racing
Role-Playing
Sandbox
Shooter
Simulation
Sports
Strategy
Visual+Novel
It took 1510.4746807000001 seconds to retrieve this data.


2022-10-15 21:21:23 [py.warnings] WARNING: C:\Users\janat\AppData\Local\Temp/ipykernel_60676/1952637497.py:292: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  games["genre"] = games["genre"].str.replace("+", " ")



62111 game records retrieved.
File saved as vgchartz-10_15_2022.csv


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

pages = 50
genres=['Action', 'Action-Adventure', 'Adventure', 'Board+Game', 'Education', 'Fighting', 'Misc', 'MMO', 'Music', 'Party', 'Platform', 'Puzzle', 'Racing', 'Role-Playing', 'Sandbox', 'Shooter', 'Simulation', 'Sports', 'Strategy', 'Visual+Novel']
rank = []
gname = []
platform = []
release_date = []
publisher = []
sales_na = []
sales_eu = []
sales_jp = []
sales_ot = []
sales_gl = []
developer= []
total_shipped=[]
game_url=[]
game_genre=[]
game_url_string=[]
count=0
fails=0

urlhead = 'http://www.vgchartz.com/games/games.php?page='
urlmid = '&results=200&name=&console=&keyword=&publisher=&genre='
urltail = '&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No'

for genre in genres:
    for page in range(1,pages):
        surl = urlhead + str(page) +urlmid +genre + urltail    
        r = requests.get(surl)
        r = r.text
        soup = BeautifulSoup(r,'html.parser')
        for row in soup.find_all('tr'):
            try:
                col=row.find_all('td')
                col_0=col[0].text
                col_4=col[4].text
                col_5=col[5].text
                col_9=col[6].text
                col_10=col[10].text
                col_11=col[11].text
                col_12=col[12].text
                col_13=col[13].text
                col_14=col[14].text
                col_15=col[15].text
                img = col[3].find('img')
                col_3=img['alt']
                a_tag=col[2].find('a')
                url_col=a_tag['href']
                col_2=(a_tag.text)
                url_string=url_col.rsplit('/', 2)[1]

                if len(col_0)<6:
                    rank.append(col_0)
                    gname.append(col_2)
                    publisher.append(col_4)
                    developer.append(col_5)
                    total_shipped.append(col_9)
                    sales_gl.append(col_10)
                    sales_na.append(col_11)
                    sales_eu.append(col_12)
                    sales_jp.append(col_13)
                    sales_ot.append(col_14)
                    release_date.append(col_15)
                    platform.append(col_3)
                    game_url.append(url_col)
                    game_genre.append(genre)
                    game_url_string.append(url_string)
                    count+=1
            except:
                fails+=1
                continue
print('vg_chartz count = '+str(count))
print('vg_chartz fails = '+str(fails))
        
columns = {'total_shipped' : total_shipped,
           'developer' : developer,
           'rank': rank,
           'name': gname,
           'platform': platform,
           'release_date': release_date,
           'publisher': publisher,
           'na_sales':sales_na,
           'eu_sales': sales_eu,
           'jp_sales': sales_jp,
           'other_sales':sales_ot,
           'global_sales':sales_gl,
           'game_genre':game_genre,
           'game_url':game_url,
           'game_url_string':game_url_string}

df = pd.DataFrame(columns)
df = df[['total_shipped','developer','rank','name','platform','release_date','publisher','na_sales','eu_sales','jp_sales','other_sales','global_sales','game_genre','game_url','game_url_string']]
df.to_csv("vgsales_full_third.csv",sep=",",encoding='utf-8')

#Rewording platforms from vgchartz wording to metacritic wording for use in url
platform_rewording_dict = {'PS3': 'playstation-3',
					   'X360': 'xbox-360',
					   'PC': 'pc',
					   'WiiU': 'wii-u',
					   '3DS': '3ds',
					   'PSV': 'playstation-vita',
					   'iOS': 'ios',
					   'Wii': 'wii',
					   'DS': 'ds',
					   'PSP': 'psp',
					   'PS2': 'playstation-2',
					   'PS': 'playstation',
					   'XB': 'xbox', 
					   'GC': 'gamecube',
					   'GBA': 'game-boy-advance',
					   'DC': 'dreamcast',
                       'PS4':'playstation-4',
                       'XOne':'xbox-one',
                       'NS':'switch'
					   }

#filtering for games that have at least 10,000 sales 
mask=df['global_sales']!='N/A'
url_search=df[mask]

#creating a list of metacritic urls  using game url string and platform to scrape for user score and further data 
meta_full_url_list=[]
meta_url= None
index=list(range(0,len(url_search)))
url_search.index=index

for row in range(0,len(url_search)):
    plat_temp=url_search.loc[row,'platform']
    url_string_temp=url_search.loc[row,'game_url_string']
    if plat_temp in platform_rewording_dict.keys():
        meta_url='https://www.metacritic.com/game/'+platform_rewording_dict[plat_temp]+'/'+url_string_temp
        meta_full_url_list.append(meta_url)
        

#=============================================================================
#Meta_critic_scraper 
#=============================================================================
meta_game_name=[]
meta_user_score=[]
meta_critic_score=[]
meta_esrb=[]
meta_full_url=[]
meta_platform=[]
meta_publisher=[]
meta_release_date=[]
meta_genre=[]
meta_error_url=[]
meta_multiplayer=[]
count=0
fails=0

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

for i in range(0,len(meta_full_url_list)):
    url= meta_full_url_list[i]
    r = requests.get(url, headers=headers)
    r = r.text
    soup = BeautifulSoup(r,'html.parser')
    critic_score_data='N/A'
    multiplayer_data='no'
    try:
        title=soup.find('div',class_='product_title')
        title_a_tag=title.find('a')
        title_data=title_a_tag.text
        
        platform=soup.find('span',class_='platform')
        platform_data=platform.text
        
        publisher_li=soup.find('li',class_="summary_detail publisher")
        publisher=publisher_li.find('span',class_='data')
        publisher_data=publisher.text
        
        release_date_li=soup.find('li',class_="summary_detail release_data")
        release_date=release_date_li.find('span',class_='data')
        release_date_data=release_date.text
        
        try:
            critic_score_a_tag = soup.find('a',class_="metascore_anchor")
            critic_score=critic_score_a_tag.find('span')
            critic_score_data=critic_score.text
        except:
            continue
            
        user_score_div_tag=soup.find('div',class_="userscore_wrap feature_userscore")
        user_score=user_score_div_tag.find('a')
        user_score_data=user_score.text
        
        rating_li=soup.find("li",class_="summary_detail product_rating")
        rating=rating_li.find('span',class_='data')
        rating_data=rating.text
        
        genre_li=soup.find('li',class_="summary_detail product_genre")
        genre=genre_li.find('span',class_='data')
        genre_data=genre.text
        
        player_li=soup.find('li',class_="summary_detail product_players")
        number=player_li.find('span',class_='data')        
        if ('1 Player' not in number)&('No Online Multiplayer' not in number):
            multiplayer_data='yes'
    
        meta_game_name.append(title_data)
        meta_platform.append(platform_data)
        meta_publisher.append(publisher_data)
        meta_release_date.append(release_date_data)
        meta_critic_score.append(critic_score_data)
        meta_user_score.append(user_score_data)
        meta_esrb.append(rating_data)
        meta_genre.append(genre_data)
        meta_full_url.append(url)
        meta_multiplayer.append(multiplayer_data)
        count+=1
        
    except:
        meta_error_url.append(url)
        fails+=1

print('metacritic count = '+str(count))
print('metacricic fails = '+str(fails))

meta_columns ={'meta_game_name':meta_game_name,
          'meta_platform':meta_platform,
          'meta_publisher':meta_publisher,
          'meta_release_date':meta_release_date,
          'meta_critic_score':meta_critic_score,
          'meta_user_score':meta_user_score,
          'meta_esrb':meta_esrb,
          'meta_genre':meta_genre,
          'meta_multiplayer':meta_multiplayer,
          'meta_full_url':meta_full_url}


meta_df = pd.DataFrame(meta_columns)
meta_df = meta_df[['meta_game_name','meta_platform','meta_publisher','meta_release_date','meta_critic_score','meta_user_score','meta_esrb','meta_genre','meta_multiplayer','meta_full_url']]
del meta_df.index.name
meta_df.to_csv("meta_full_third.csv",sep=",",encoding='utf-8')

#=============================================================================
#IGDB SCRAPER 
#=============================================================================


count=0
none=0
fails=0
igdb_game_string=[]
igdb_series=[]


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


for row in range(0,len(url_search)):
    try:
        urlhead= 'https://www.igdb.com/games/'
        urltail=url_search.loc[row,'game_url_string']
        url=urlhead+urltail
        r = requests.get(url, headers=headers)
        r = r.text
        soup = BeautifulSoup(r,'html.parser')
        soup=soup.text
        try:
            if ('Series:' in soup):
                igdb_series.append('yes')
                igdb_game_string.append(urltail)
                count+=1
            else:
                igdb_series.append('no')
                igdb_game_string.append(urltail)
                none+=1   
        except:     
            fails+=1
            continue
    except:
        fails+=1
        continue
    
print('igdb mp = '+str(count))
print('igdb no_mp = '+str(none))
print('igdb fails = '+str(fails))

    
igdb_columns ={'igdb_game_string':igdb_game_string,
          'igdb_series':igdb_series}

igdb_df = pd.DataFrame(igdb_columns)
igdb_df = igdb_df[['igdb_game_string','igdb_series']]
del igdb_df.index.name
igdb_df.to_csv("igdb_full_third.csv",sep=",",encoding='utf-8')

ConnectionError: HTTPSConnectionPool(host='www.vgchartz.com', port=443): Max retries exceeded with url: /games/games.php?page=9&results=200&name=&console=&keyword=&publisher=&genre=Action-Adventure&order=Sales&ownership=Both&boxart=Both&banner=Both&showdeleted=&region=All&goty_year=&developer=&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=1&showreleasedate=1&showlastupdate=1&showvgchartzscore=1&showcriticscore=1&showuserscore=1&showshipped=1&alphasort=&showmultiplat=No (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001903DBA9B50>: Failed to establish a new connection: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu'))

In [6]:
import requests
from bs4 import BeautifulSoup
import json
from scripts import DataHelper, send_email
import datetime 

url = 'https://www.metacritic.com/browse/games/release-date/available/ps4/date'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
     (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

games = soup.find_all("div", {"class":"product_wrap"})[:-3]

# scrape the name, release date, user score and critic score of each game and store them in a dictionary
game_data = []
for game in games:
    
    title = game.find("div", {"class":"basic_stat product_title"})

    for tag in game.find_all("li", {"class":"stat release_date"}):
        release_date = tag.find("span", {"class":"data"})
     
    critic_score = game.find_all("div", {"class":"basic_stat product_score brief_metascore"})[0]
    user_score = game.find_all("span")[1]
    pair = {
          "title":title.text.strip(),
          "release_date":release_date.text.strip(),
          "critic_score":critic_score.text.strip(),
          "user_score":user_score.text.strip()
     }
    game_data.append(pair)

# save the dictionary as JSON so the DataHelper can receive and convert it into dataframe
with open("data.json", "w") as f:
     json.dump(game_data, f, indent=2)

# create an instance of DataHelper 
dh = DataHelper()

# perform operations of DataHelper 
test_data = dh.load_and_clean()
complete_data = dh.get_complete_data(test_data, save_csv=True)
o_best = dh.store_overall_best(complete_data)
u_best = dh.store_users_best(complete_data)
c_best = dh.store_critic_best(complete_data)
contr_good = dh.store_controverial_good(complete_data)
contr_bad = dh.store_controverial_bad(complete_data)


# prompt the user to put email address to send email to
from_address = input("Sender's Email address: ")
from_password = input("Sender's password: ")
to_address = input("Email address to send this email to:  ")

send_email(from_address, from_password, subject=f"Metacritic Scraper {datetime.date.today()}", content=o_best + "\n" + u_best\
          + "\n" + c_best + "\n" + contr_good + "\n" + contr_bad, to_address=to_address)

ImportError: cannot import name 'DataHelper' from 'scripts' (unknown location)